# True Zonal Mean

Calculate the *true zonal mean* of a scalar quantity regardless of the horizontal mesh. 

Specifically, we calculate the volume weighted mean along all grid cells whose centres fall within finite latitude intervals rather than the arithmetic mean of cells along the model's curvilinear grid. The method presented can also be used to re-grid models onto the same latitudinal grid and the general principles can be used to define any multidimensional sum or average using the `xhistogram` package.
    
**Requirements:** 
Select the `conda/analysis3-22.04` (or later) kernel.
This code should work for just about any MOM5 configuration since all we are grabbing is temeprature and standard grid information. You can swap temperature with any other scalar variable. You can also in principle swap latitude with another scalar.

In [1]:
import intake
import matplotlib.pyplot as plt
import cmocean as cm
import xarray as xr
import numpy as np
from dask.distributed import Client
from xhistogram.xarray import histogram #This package has only recently been added to conda/analysis3-20.01

In [2]:
client = Client(threads_per_worker = 1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 28
Total threads: 28,Total memory: 125.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37279,Workers: 28
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 125.19 GiB
Comm: tcp://127.0.0.1:43853,Total threads: 1
Dashboard: /proxy/38613/status,Memory: 4.47 GiB
Nanny: tcp://127.0.0.1:44461,


Open ACCESS-NRI's default catalog:

In [3]:
catalog = intake.cat.access_nri

Choose the experiment and the variable we want to average. This example uses temperature but you can choose any scalar, 3D variable:

In [4]:
catalog.search(name='.*025deg_jra55.*')

,model,description,realm,frequency,variable
name,,,,,
025deg_jra55_iaf_era5comparison,{ACCESS-OM2},{0.25 degree ACCESS-OM2 global model configuration with JRA55-do v1.5.0\ninterannual forcing (1980-2019)},"{ocean, seaIce}","{1day, 1mon, fx}","{ke_tot, strtltx_m, ULON, strcory_m, ty_trans_int_z, total_ocean_sens_heat, salt, sfc_salt_flux_ice, snow_ai_m, trsig_m, ty_trans_nrho_submeso, u, frzmlt_m, geolat_c, tx_trans_int_z, fswabs_ai_m, ..."
025deg_jra55_iaf_omip2_cycle1,{ACCESS-OM2},{Cycle 1/6 of 0.25 degree ACCESS-OM2 physics-only global configuration with JRA55-do v1.4 OMIP2 interannual forcing (1958-2019)},"{ocean, seaIce}","{1day, 1mon, fx, 1yr}","{ke_tot, bmf_v, ULON, strcory_m, strtltx_m, sfc_salt_flux_ice, total_ocean_sens_heat, salt, ty_trans_int_z, ty_trans_gm, neutral_diffusion_temp, alidf_ai_m, snow_ai_m, ty_trans_nrho_submeso, trsig..."
025deg_jra55_iaf_omip2_cycle2,{ACCESS-OM2},{Cycle 1/6 of 0.25 degree ACCESS-OM2 physics-only global configuration with JRA55-do v1.4 OMIP2 interannual forcing (1958-2019)},"{ocean, seaIce}","{1day, 1mon, fx, 1yr}","{ke_tot, bmf_v, ULON, strcory_m, strtltx_m, sfc_salt_flux_ice, total_ocean_sens_heat, salt, ty_trans_int_z, ty_trans_gm, neutral_diffusion_temp, alidf_ai_m, snow_ai_m, ty_trans_nrho_submeso, trsig..."
025deg_jra55_iaf_omip2_cycle3,{ACCESS-OM2},{Cycle 3/6 of 0.25 degree ACCESS-OM2 physics-only global configuration with JRA55-do v1.4 OMIP2 interannual forcing (1958-2019)},"{ocean, seaIce}","{1day, 1mon, fx, 1yr}","{ke_tot, bmf_v, ULON, strcory_m, strtltx_m, sfc_salt_flux_ice, total_ocean_sens_heat, salt, ty_trans_int_z, ty_trans_gm, neutral_diffusion_temp, alidf_ai_m, snow_ai_m, ty_trans_nrho_submeso, trsig..."
025deg_jra55_iaf_omip2_cycle4,{ACCESS-OM2},{Cycle 4/6 of 0.25 degree ACCESS-OM2 physics-only global configuration with JRA55-do v1.4 OMIP2 interannual forcing (1958-2019)},"{ocean, seaIce}","{1day, 1mon, fx, 1yr}","{ke_tot, bmf_v, ULON, strcory_m, strtltx_m, sfc_salt_flux_ice, total_ocean_sens_heat, salt, ty_trans_int_z, ty_trans_gm, neutral_diffusion_temp, alidf_ai_m, snow_ai_m, ty_trans_nrho_submeso, trsig..."
025deg_jra55_iaf_omip2_cycle5,{ACCESS-OM2},{Cycle 5/6 of 0.25 degree ACCESS-OM2 physics-only global configuration with JRA55-do v1.4 OMIP2 interannual forcing (1958-2019)},"{ocean, seaIce}","{1day, 1mon, fx, 1yr}","{ke_tot, bmf_v, ULON, strcory_m, strtltx_m, sfc_salt_flux_ice, total_ocean_sens_heat, salt, ty_trans_int_z, ty_trans_gm, neutral_diffusion_temp, alidf_ai_m, snow_ai_m, ty_trans_nrho_submeso, trsig..."
025deg_jra55_iaf_omip2_cycle6,{ACCESS-OM2},{Cycle 6/6 of 0.25 degree ACCESS-OM2 physics-only global configuration with JRA55-do v1.4 OMIP2 interannual forcing (1958-2019)},"{ocean, seaIce}","{1day, 1mon, fx}","{ke_tot, bmf_v, ULON, strcory_m, strtltx_m, sfc_salt_flux_ice, total_ocean_sens_heat, salt, ty_trans_int_z, ty_trans_gm, neutral_diffusion_temp, alidf_ai_m, snow_ai_m, ty_trans_nrho_submeso, trsig..."
025deg_jra55_ryf9091_gadi,{ACCESS-OM2},{0.25 degree ACCESS-OM2 physics-only global configuration with JRA55-do v1.3 RYF9091 repeat year forcing (May 1990 to Apr 1991)},"{ocean, seaIce}","{1mon, fx, 1yr}","{ke_tot, strtltx_m, ULON, strcory_m, ty_trans_int_z, salt, total_ocean_sens_heat, sfc_salt_flux_ice, vocn_m, alidf_ai_m, snow_ai_m, trsig_m, u, frzmlt_m, geolat_c, tx_trans_int_z, fswabs_ai_m, sal..."
025deg_jra55_ryf_era5comparison,{ACCESS-OM2},{0.25 degree ACCESS-OM2 global model configuration with JRA55-do v1.4.0\nRYF9091 repeat year forcing (May 1990 to Apr 1991)},"{ocean, seaIce}","{1day, 1mon, fx}","{ke_tot, bmf_v, ULON, strcory_m, strtltx_m, ty_trans_int_z, total_ocean_sens_heat, salt, sfc_salt_flux_ice, alidf_ai_m, snow_ai_m, trsig_m, ty_trans_nrho_submeso, u, frzmlt_m, geolat_c, tx_trans_i..."


In [5]:
experiment = '025deg_jra55_iaf_omip2_cycle2' #Choose any run
variable = 'temp' #any scalar variable where volume weighted averaging makes sense

In [6]:
cat_subset = catalog[experiment]
var_search = cat_subset.search(variable=variable)
darray = var_search.to_dask()
variable_to_average = darray[variable].mean('time')

First we show the standard approach, which is to take the arithmetic mean of all grid cells along the quasi-longitudinal coordinate. For MOM5's tri-polar grid this approach is in principle "okay" for the southern hemisphere, where grid cell areas are constant at fixed latitude. It doesn't though, take into account partial cells.

The `xarray`'s method `.mean(dim='dimension')` applies `numpy.mean()` across that dimension. This is simply the arithmetic mean.

For some scalar $T$ the arithmetic mean, e.g., across dimension `i`, is given by

$$ \left<T\right>_{j,k} = \frac{1}{I}\sum_{i=1}^{I} T_{i,j,k},$$

where $i$, $j$ and $k$ are the indicies in the $x$, $y$ and $z$ directions respectively of the curvilinear grid and $I$ is the number of indicies along the $x$ axis. 

In [7]:
zonal_arith_mean = variable_to_average.mean(dim='xt_ocean')

Load to speed up the plotting:

In [ ]:
%%time
zonal_arith_mean = zonal_arith_mean.load()

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 40.92 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [ ]:
plt.figure(figsize=(10, 5))
zonal_arith_mean.plot(yincrease=False, vmin=273, vmax=300, cmap=cm.cm.thermal)
plt.title('Zonal arithmetic mean');

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 40.92 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


The main issue with this average is that the 'latitude' coordinate may be meaningless near the north pole, particularly when comparing to observational analyses or other models which can have either a regular grid or a different curvilinear grid. Even different versions of MOM might have different grids! 

Let us consider what the true zonal average looks like. That is consider a set of latitude 'edges' $\{\phi'_{1/2},\phi'_{1+1/2},...,\phi'_{\ell-1/2},\phi'_{\ell+1/2},...,\phi'_{L+1/2}\}$ between which we want to compute an average of $T$ at $\{\phi'_{1},\phi'_{2},...,\phi'_{\ell},...,\phi'_{L}\}$ such that

$$ \overline{T}(\phi'_\ell,\sigma) = \dfrac{\iint_{\phi'_{\ell-1/2} < \phi \leq \phi'_{\ell+1/2}} T(\phi,\lambda,\sigma)\frac{\partial z}{\partial \sigma}(\phi,\lambda,\sigma)\,\mathrm{d}A}{\iint_{\phi'_{\ell-1/2} < \phi \leq \phi'_{\ell+1/2}}\frac{\partial z}{\partial \sigma}(\phi,\lambda,\sigma)\,\mathrm{d}A},$$

where $\lambda$ is longitude and $\sigma$ is an arbitrary vertical coordinate. 

In discrete form this average is

$$\overline{T}_{\ell,k} = \frac{\sum_{i=1}^{I}\sum_{j=1}^{J}\delta_{i,j}T_{i,j,k}\Delta Z_{i,j,k}\Delta \mathrm{Area}_{i,j}}{\sum_{i=1}^{I}\sum_{j=1}^{J}\delta_{i,j,k}\Delta Z_{i,j,k}\Delta \mathrm{Area}_{i,j}},$$

where $\delta_{i,j} = 1$ if $\phi'_{\ell-1/2}<\phi_{i,j}\leq \phi'_{\ell+1/2}$ and $\delta_{i,j} = 0$ elsewhere, $\Delta Z$ is the grid cell vertical thickness and $\Delta \mathrm{Area}$ is the grid cell horizontal area.

For our purposes we will use the edges of the models `xt_ocean` coordinate to define $\phi'_{\ell+1/2}$ so the number of 'bins' $L$ will be the same as the length of the quasi-latitude coordinate ($J$). 

Fortunately, as you can see below, the two sums are weighted histograms (one for $T$ times volume and the other for just volume) and these can be rapidly computed using `xhistogram`.

First lets load the scalar variable (latitude) we want to use as our coordinate then define the bin edges.

In [ ]:
coord = 'geolat_t' #can be any scalar (2D, 3D, eulerian, lagrangian etc)
cat_subset = catalog[experiment]
var_search = cat_subset.search(variable=coord)
var_search = var_search.search(path=var_search.df['path'][0])
darray = var_search.to_dask()
darray = darray[coord]
variable_as_coord = darray #might need ncfile='ocean.nc' if a tracer

Now we want to define the coordinate bins as the latitude edges of the t-cells, adding the first edge (0) at latitude -90:

In [ ]:
#Define the coordinate bins as the latitude edges of the T-cells
cat_subset = catalog[experiment]
var_search = cat_subset.search(variable='yu_ocean')
var_search = var_search.search(path=var_search.df['path'][0])
darray = var_search.to_dask()
darray = darray['yu_ocean']
yu_ocean = darray
#make numpy array (using .values) and add 1st edge at -90
bins =  np.insert(yu_ocean.values, 0, np.array(-90), axis=0) 

In [ ]:
#Alternatively we could just use some regular grid like this 
#coordbins =  np.linspace(-80, 90, 50)
#or use a grid from a different (coarser) model.

cat_subset = catalog[experiment]
var_search = cat_subset.search(variable='dzt')
darray = var_search.to_dask()
darray = darray['dzt']
dzt = darray #thickness of cells
cat_subset = catalog[experiment]
var_search = cat_subset.search(variable='area_t')
var_search = var_search.search(path=var_search.df['path'][0])
darray = var_search.to_dask()
darray = darray['area_t']
area_t = darray #area of cells

dVol = dzt * area_t #Volume of cells

Now let's compute the numerator and denominator of the equation above using `xhistogram`, then the time mean and then the zonal mean.

In [ ]:
histVolCoordDepth = histogram(variable_as_coord.broadcast_like(dVol).where(~np.isnan(dVol)), bins=[bins], weights=dVol, dim=['yt_ocean', 'xt_ocean'])
histTVolCoordDepth = histogram(variable_as_coord.broadcast_like(dVol).where(~np.isnan(dVol)), bins=[bins], weights=dVol * variable_to_average, dim=['yt_ocean', 'xt_ocean'])
coord_mean = (histTVolCoordDepth/histVolCoordDepth).groupby('time.year').mean(dim='time')

Now we can plot the results which thankfully retain all the data-array info on variables and axis etc.

In [ ]:
plt.figure(figsize=(10, 5))
coord_mean.sel(year=2000).plot(yincrease=False, vmin=273, vmax=300, cmap='Oranges')
plt.title('True zonal mean');

Since we used the same bin edges as the standard `yt_ocean` coordinate we can take a difference between the arithmetic mean along the model's x-axis and our mean along grid cells within latitude bands. The main differences are near the North Pole where the grid is furthest for being regular. There are also differences near the Antacrtic Shelf suggesting partial cells also matter.

In [ ]:
zonal_minus_x_mean = coord_mean.sel(year=2000) - x_arith_mean.sel(year=2000).values

plt.figure(figsize=(10, 5))
zonal_minus_x_mean.plot(yincrease=False, vmin=-0.8, vmax=0.8, cmap='RdBu_r', extend='both')
plt.title('True zonal minus $x$-coordinate arithmetic mean');

`xarray` has a new `weighted` functionality which allows it to do weighted means instead of arithmetic mean.

Let's see how that works out... We chose `dVol` as the weights and we only do the comparisson for year 2000.

In [ ]:
variable_to_average_weighted = variable_to_average.copy().sel(time='2000').mean(dim='time')
variable_to_average_weighted = variable_to_average_weighted.weighted(dVol.sel(time='2000').fillna(0))
meanweighted_y2000 = variable_to_average_weighted.mean(dim='xt_ocean').groupby('time.year').mean(dim='time').sel(year=2000)

In [ ]:
zonal_minus_x_mean = coord_mean.sel(year=2000) - meanweighted_y2000.values

plt.figure(figsize=(10, 5))
zonal_minus_x_mean.plot(yincrease=False, vmin=-0.8, vmax=0.8, cmap='RdBu_r')
plt.title("True zonal minus xarray's $x$-coordinate weighted mean");    

South of 65N, where complications of the tripolar grid don't matter, `xarray`'s weighted mean does the job! But in the region of the tripolar we need to be more careful.